# 통합 식품영양성분DB 정리
### - [클리닝](#1.-클리닝)
### - [타입 변경(object -> float)](#2.-타입-변경)
### - [데이터 단위 통일(영양소 g으로 통일)](#3.-영양소-별-데이터-단위-통일)
### - [각 식품의 영양소 최대 값 행 추가](#4.-)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re


In [2]:
food_nutrients = pd.read_excel('통합 식품영양성분DB_20211106.xlsx', skiprows=[0,1,2])
food_nutrients.head()

,NO,SAMPLE_ID,식품코드,DB군,상용제품,식품명,연도,지역 / 제조사,채취시기,식품대분류,...,냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),폐기율(%),가식부(%),산가용성물질(%),카페인(㎎),성분표출처,발행기관
0,1,D000006-94-AVG,D000006,음식,품목대표,꿩불고기,2019,충주,평균,구이류,...,-,-,-,5.8,-,-,-,-,식약처('16) 제4권,식품의약품안전처
1,2,D000007-ZZ-AVG,D000007,음식,품목대표,닭갈비,2019,전국(대표),평균,구이류,...,-,-,-,7,-,-,-,-,식약처('16) 제4권,식품의약품안전처
2,3,D000008-66-AVG,D000008,음식,품목대표,닭갈비,2019,춘천,평균,구이류,...,-,-,-,4.5,-,-,-,-,식약처('16) 제4권,식품의약품안전처
3,4,D000009-ZZ-AVG,D000009,음식,품목대표,닭꼬치,2019,전국(대표),평균,구이류,...,-,-,-,1.2176954799497,-,-,-,-,식약처('12) 제1권,식품의약품안전처
4,5,D000010-ZZ-AVG,D000010,음식,품목대표,더덕구이,2019,전국(대표),평균,구이류,...,-,-,-,2.5,-,-,-,-,식약처('17) 제5권,식품의약품안전처


In [3]:
pd.options.display.max_columns = 241
pd.set_option('display.max.colwidth', 50)

food_nutrients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58597 entries, 0 to 58596
Columns: 241 entries, NO to 발행기관
dtypes: float64(1), int64(2), object(238)
memory usage: 107.7+ MB


## 1. 클리닝 

#### 필요없는 colum 제거 및 float로 변환하기 위해 데이터 정제

In [4]:
food_nutrients = food_nutrients.drop(['SAMPLE_ID', '식품코드', '연도', '지역 / 제조사',\
                                      '채취시기', '폐기율(%)', '성분표출처', '발행기관'], axis = 1)

In [5]:
food_nutrients.head()

,NO,DB군,상용제품,식품명,식품대분류,식품상세분류,1회제공량,내용량_단위,총내용량(g),총내용량(mL),에너지(㎉),에너지(kj),수분(g),수분(%),단백질(g),지방(g),지질-가식부 100g당,탄수화물(g),총당류(g),자당(g),포도당(g),과당(g),유당(g),맥아당(g),갈락토오스(g),당알콜(g),에리스리톨(g),총 식이섬유(g),총 식이섬유(mg),총 식이섬유(%),수용성 식이섬유(g),불용성 식이섬유(g),셀룰로오스(%),리그닌(%),칼슘(㎎),철(㎎),철(㎍),마그네슘(㎎),인(㎎),칼륨(g),칼륨(㎎),나트륨(㎎),아연(㎎),구리(㎎),구리(㎍),망간(㎎),망간(㎍),셀레늄(㎍),몰리브덴(㎍),요오드(㎍),염소(㎎),비타민 A(㎍),비타민 A(㎍ RE),레티놀(㎍),베타카로틴(㎍),레티놀A효능,비타민 D(D2+D3)(㎍),비타민 D2(㎍),비타민 D3(㎍),비타민 D1(㎍),비타민 E(㎎),비타민 E(㎎ α-TE),알파 토코페롤(㎎),베타 토코페롤(㎎),감마 토코페롤(㎎),델타 토코페롤(㎎),알파 토코트리에놀(㎎),베타 토코트리에놀(㎎),감마 토코트리에놀(㎎),델타 토코트리에놀(㎎),토코페롤(㎎),토코트리에놀(㎎),비타민 K(㎎),비타민 K(㎍),비타민 K1(㎍),비타민 K2(㎍),비타민 B1(㎎),비타민 B1(㎍),비타민 B2(㎎),비타민 B2(㎍),나이아신(㎎),나이아신(㎎ NE),나이아신(NE)(㎎),나이아신(NE)(㎎ NE),니코틴산 (㎎),니코틴아마이드(㎎),판토텐산(㎎),판토텐산(㎍),비타민 B6(㎎),비타민 B6(㎍),피리독신(㎎),비오틴(㎍),엽산(DFE)(㎍),엽산 - 천연 엽산(㎍),엽산 - 합성 엽산(㎍),비타민 B12(㎎),비타민 B12(㎍),비타민 C(g),비타민 C(㎎),콜린(g),콜린(㎎),총 아미노산(g),총 아미노산(㎎),필수 아미노산(㎎),이소류신(㎎),류신(㎎),라이신(㎎),메티오닌(㎎),페닐알라닌(㎎),트레오닌(㎎),트립토판(㎎),발린(㎎),히스티딘(㎎),아르기닌(㎎),비필수 아미노산(㎎),티로신(㎎),시스테인(㎎),알라닌(㎎),아스파르트산(㎎),글루탐산(㎎),글리신(㎎),프롤린(㎎),세린(㎎),타우린(㎎),글리신 베타인 (㎎),호마린(㎎),트리고넬린(㎎),리보핵산(㎎),데옥시리보핵산(㎎),콜레스테롤(g),콜레스테롤(㎎),총 지방산(g),총 필수 지방산(g),총 포화 지방산(g),총 포화 지방산(%),부티르산(4:0)(g),부티르산(4:0)(㎎),카프로산(6:0)(g),카프로산(6:0)(㎎),카프릴산(8:0)(g),카프릴산(8:0)(㎎),카프르산(10:0)(g),카프르산(10:0)(㎎),라우르산(12:0)(g),라우르산(12:0)(㎎),라우르산(12:0)(%),트라이데칸산(13:0)(㎎),미리스트산(14:0)(g),미리스트산(14:0)(㎎),미리스트산(14:0)(%),펜타데칸산(15:0)(㎎),팔미트산(16:0)(g),팔미트산(16:0)(㎎),팔미트산(16:0)(%),헵타데칸산(17:0)(㎎),스테아르산(18:0)(g),스테아르산(18:0)(㎎),스테아르산(18:0)(%),아라키드산(20:0)(g),아라키드산(20:0)(㎎),아라키드산(20:0)(%),헨에이코산산(21:0)(㎎),베헨산(22:0)(㎎),트리코산산(23:0)(㎎),리그노세르산(24:0)(㎎),총 단일 불포화지방산(g),총 단일 불포화지방산(%),미리스톨레산(14:1)(g),미리스톨레산(14:1)(㎎),미리스톨레산(14:1)(%),팔미톨레산(16:1)(g),팔미톨레산(16:1)(㎎),팔미톨레산(16:1)(%),헵타데센산(17:1)(㎎),올레산(18:1(n-9))(g),올레산(18:1(n-9))(㎎),올레산(18:1(n-9))(%),박센산(18:1(n-7))(g),박센산(18:1(n-7))(㎎),가돌레산(20:1)(g),가돌레산(20:1)(㎎),가돌레산(20:1)(%),에루크산(22:1)(㎎),에루크산(22:1)(%),네르본산(24:1)(㎎),총 다중 불포화지방산(g),총 다중 불포화지방산(%),리놀레산(18:2(n-6)c)(g),리놀레산(18:2(n-6)c)(㎎),리놀레산(18:2(n-6)c)(%),알파 리놀렌산(18:3(n-3))(g),알파 리놀렌산(18:3(n-3))(㎎),리놀렌산(18:3(n-3)(%),감마 리놀렌산(18:3(n-6))(g),감마 리놀렌산(18:3(n-6))(㎎),스테아리돈산(18:4)(%),에이코사디에노산(20:2(n-6))(g),에이코사디에노산(20:2(n-6))(㎎),에이코사트리에노산(20:3(n-3))(㎎),에이코사트리에노산(20:3(n-6))(g),에이코사트리에노산(20:3(n-6))(㎎),아라키돈산(20:4(n-6))(g),아라키돈산(20:4(n-6))(㎎),아라키돈산(20:4(n-6))(%),에이코사테트라에노산(20:4(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(g),에이코사펜타에노산(20:5(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(%),도코사디에노산(22:2)(㎎),도코사펜타에노산(22:5(n-3))(g),도코사펜타에노산(22:5(n-3))(㎎),도코사펜타에노산(22:5(n-3))(%),도코사헥사에노산(22:6(n-3))(g),도코사헥사에노산(22:6(n-3))(㎎),도코사헥사에노산(22:6(n-3))(%),EPA와 DHA의 합(㎎),오메가 3 지방산(g),오메가 6 지방산(g),트랜스 지방산(g),트랜스 올레산(18:1(n-9)t)(g),트랜스 올레산(18:1(n-9)t)(㎎),트랜스 리놀레산 (18:2t)(g),트랜스 리놀레산 (18:2t)(㎎),트랜스 리놀렌산(18:3t)(g),트랜스 리놀렌산(18:3t)(㎎),트랜스 리놀렌산(18:3t)(%),냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),가식부(%),산가용성물질(%),카페인(㎎)
0,1,음식,품목대표,꿩불고기,구이류,육류구이,500.0,g,-,-,368.8,-,412.6,-,33.5,8.5,-,39.7,16.9,7.2,2.8,2.8,0.7,3.5,-,-,-,9.8,-,-,-,-,-,-,105.61,-,4,85.39,458.05,-,1243.12,1264.31,3.99,0.32,-,0.68,-,47.55,-,-,-,-,-,0,1424.58,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,8.73,0.06,-,-,-,-,0,-,0.33,-,3.61,-,-,-,-,-,-,-,-,-,-,-,63.13,-,-,-,0,-,2.99,-,-,-,-,-,1284.37582123574,2328.59892766839,2484.75047814224,645.190070073169,1274.92730261488,1410.84127771718,-,1367.68831341948,848.539312406619,2205.5847774439,-,919.805669181432,276.498720836072,1642.38119022329,2781.77304833044,4784.60634708081,1419.92479570739,1092.29548260871,1264.95795056827,-,-,-,-,-,-,-,106.18,-,-,1.9,-,0,-,0,-,0,-,0,-,0.00711483471961893,-,-,-,0.0317835536096363,-,-,-,1.28384114493856,-,-,-,0.527395267901326,-,-,0.0290123491481805,-,-,-,-,-,-,-,-,0,-,-,0.0813516573054785,-,-,-,2.30772104748742,-,-,0.113462530850897,-,0.0279762322988018,-,-,-,-,-,-,-,3.23601791813763,-,-,0.656722581376586,-,-,0.018708284652282,-,-,0.00664887007716417,-,-,0,-,0.142840481460645,-,-,-,0,-,-,-,0,-,-,0.0234269823295255,-,-,-,-,-,0.1,0.018116327971628,-,0.0534160839712029,-,0,-,-,-,-,-,5.8,-,-,-
1,2,음식,품목대표,닭갈비,구이류,육류구이,400.0,g,-,-,595.61,-,276.4,-,45.9,25.8,

In [6]:
food_nutrients_copy = food_nutrients.copy()
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = food_nutrients.loc[: ,'단백질(g)':'카페인(㎎)'].replace(['-', 'tr', 'Tr'], 0)

In [7]:
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = \
        food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].replace(re.compile(r'미만'), 0)
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = \
        food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].replace(re.compile(r'이하'), 0)
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = \
        food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].replace(re.compile(r'<'), 0)

food_nutrients_copy.head()

,NO,DB군,상용제품,식품명,식품대분류,식품상세분류,1회제공량,내용량_단위,총내용량(g),총내용량(mL),에너지(㎉),에너지(kj),수분(g),수분(%),단백질(g),지방(g),지질-가식부 100g당,탄수화물(g),총당류(g),자당(g),포도당(g),과당(g),유당(g),맥아당(g),갈락토오스(g),당알콜(g),에리스리톨(g),총 식이섬유(g),총 식이섬유(mg),총 식이섬유(%),수용성 식이섬유(g),불용성 식이섬유(g),셀룰로오스(%),리그닌(%),칼슘(㎎),철(㎎),철(㎍),마그네슘(㎎),인(㎎),칼륨(g),칼륨(㎎),나트륨(㎎),아연(㎎),구리(㎎),구리(㎍),망간(㎎),망간(㎍),셀레늄(㎍),몰리브덴(㎍),요오드(㎍),염소(㎎),비타민 A(㎍),비타민 A(㎍ RE),레티놀(㎍),베타카로틴(㎍),레티놀A효능,비타민 D(D2+D3)(㎍),비타민 D2(㎍),비타민 D3(㎍),비타민 D1(㎍),비타민 E(㎎),비타민 E(㎎ α-TE),알파 토코페롤(㎎),베타 토코페롤(㎎),감마 토코페롤(㎎),델타 토코페롤(㎎),알파 토코트리에놀(㎎),베타 토코트리에놀(㎎),감마 토코트리에놀(㎎),델타 토코트리에놀(㎎),토코페롤(㎎),토코트리에놀(㎎),비타민 K(㎎),비타민 K(㎍),비타민 K1(㎍),비타민 K2(㎍),비타민 B1(㎎),비타민 B1(㎍),비타민 B2(㎎),비타민 B2(㎍),나이아신(㎎),나이아신(㎎ NE),나이아신(NE)(㎎),나이아신(NE)(㎎ NE),니코틴산 (㎎),니코틴아마이드(㎎),판토텐산(㎎),판토텐산(㎍),비타민 B6(㎎),비타민 B6(㎍),피리독신(㎎),비오틴(㎍),엽산(DFE)(㎍),엽산 - 천연 엽산(㎍),엽산 - 합성 엽산(㎍),비타민 B12(㎎),비타민 B12(㎍),비타민 C(g),비타민 C(㎎),콜린(g),콜린(㎎),총 아미노산(g),총 아미노산(㎎),필수 아미노산(㎎),이소류신(㎎),류신(㎎),라이신(㎎),메티오닌(㎎),페닐알라닌(㎎),트레오닌(㎎),트립토판(㎎),발린(㎎),히스티딘(㎎),아르기닌(㎎),비필수 아미노산(㎎),티로신(㎎),시스테인(㎎),알라닌(㎎),아스파르트산(㎎),글루탐산(㎎),글리신(㎎),프롤린(㎎),세린(㎎),타우린(㎎),글리신 베타인 (㎎),호마린(㎎),트리고넬린(㎎),리보핵산(㎎),데옥시리보핵산(㎎),콜레스테롤(g),콜레스테롤(㎎),총 지방산(g),총 필수 지방산(g),총 포화 지방산(g),총 포화 지방산(%),부티르산(4:0)(g),부티르산(4:0)(㎎),카프로산(6:0)(g),카프로산(6:0)(㎎),카프릴산(8:0)(g),카프릴산(8:0)(㎎),카프르산(10:0)(g),카프르산(10:0)(㎎),라우르산(12:0)(g),라우르산(12:0)(㎎),라우르산(12:0)(%),트라이데칸산(13:0)(㎎),미리스트산(14:0)(g),미리스트산(14:0)(㎎),미리스트산(14:0)(%),펜타데칸산(15:0)(㎎),팔미트산(16:0)(g),팔미트산(16:0)(㎎),팔미트산(16:0)(%),헵타데칸산(17:0)(㎎),스테아르산(18:0)(g),스테아르산(18:0)(㎎),스테아르산(18:0)(%),아라키드산(20:0)(g),아라키드산(20:0)(㎎),아라키드산(20:0)(%),헨에이코산산(21:0)(㎎),베헨산(22:0)(㎎),트리코산산(23:0)(㎎),리그노세르산(24:0)(㎎),총 단일 불포화지방산(g),총 단일 불포화지방산(%),미리스톨레산(14:1)(g),미리스톨레산(14:1)(㎎),미리스톨레산(14:1)(%),팔미톨레산(16:1)(g),팔미톨레산(16:1)(㎎),팔미톨레산(16:1)(%),헵타데센산(17:1)(㎎),올레산(18:1(n-9))(g),올레산(18:1(n-9))(㎎),올레산(18:1(n-9))(%),박센산(18:1(n-7))(g),박센산(18:1(n-7))(㎎),가돌레산(20:1)(g),가돌레산(20:1)(㎎),가돌레산(20:1)(%),에루크산(22:1)(㎎),에루크산(22:1)(%),네르본산(24:1)(㎎),총 다중 불포화지방산(g),총 다중 불포화지방산(%),리놀레산(18:2(n-6)c)(g),리놀레산(18:2(n-6)c)(㎎),리놀레산(18:2(n-6)c)(%),알파 리놀렌산(18:3(n-3))(g),알파 리놀렌산(18:3(n-3))(㎎),리놀렌산(18:3(n-3)(%),감마 리놀렌산(18:3(n-6))(g),감마 리놀렌산(18:3(n-6))(㎎),스테아리돈산(18:4)(%),에이코사디에노산(20:2(n-6))(g),에이코사디에노산(20:2(n-6))(㎎),에이코사트리에노산(20:3(n-3))(㎎),에이코사트리에노산(20:3(n-6))(g),에이코사트리에노산(20:3(n-6))(㎎),아라키돈산(20:4(n-6))(g),아라키돈산(20:4(n-6))(㎎),아라키돈산(20:4(n-6))(%),에이코사테트라에노산(20:4(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(g),에이코사펜타에노산(20:5(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(%),도코사디에노산(22:2)(㎎),도코사펜타에노산(22:5(n-3))(g),도코사펜타에노산(22:5(n-3))(㎎),도코사펜타에노산(22:5(n-3))(%),도코사헥사에노산(22:6(n-3))(g),도코사헥사에노산(22:6(n-3))(㎎),도코사헥사에노산(22:6(n-3))(%),EPA와 DHA의 합(㎎),오메가 3 지방산(g),오메가 6 지방산(g),트랜스 지방산(g),트랜스 올레산(18:1(n-9)t)(g),트랜스 올레산(18:1(n-9)t)(㎎),트랜스 리놀레산 (18:2t)(g),트랜스 리놀레산 (18:2t)(㎎),트랜스 리놀렌산(18:3t)(g),트랜스 리놀렌산(18:3t)(㎎),트랜스 리놀렌산(18:3t)(%),냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),가식부(%),산가용성물질(%),카페인(㎎)
0,1,음식,품목대표,꿩불고기,구이류,육류구이,500.0,g,-,-,368.8,-,412.6,-,33.5,8.5,0,39.7,16.9,7.2,2.8,2.8,0.7,3.5,0,0,0,9.8,0,0,0,0,0,0,105.61,0,4,85.39,458.05,0,1243.12,1264.31,3.99,0.32,0,0.68,0,47.55,0,0,0,0,0,0,1424.58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.73,0.06,0,0,0,0,0,0,0.33,0,3.61,0,0,0,0,0,0,0,0,0,0,0,63.13,0,0,0,0,0,2.99,0,0,0,0,0,1284.37582123574,2328.59892766839,2484.75047814224,645.190070073169,1274.92730261488,1410.84127771718,0,1367.68831341948,848.539312406619,2205.5847774439,0,919.805669181432,276.498720836072,1642.38119022329,2781.77304833044,4784.60634708081,1419.92479570739,1092.29548260871,1264.95795056827,0,0,0,0,0,0,0,106.18,0,0,1.9,0,0,0,0,0,0,0,0,0,0.00711483471961893,0,0,0,0.0317835536096363,0,0,0,1.28384114493856,0,0,0,0.527395267901326,0,0,0.0290123491481805,0,0,0,0,0,0,0,0,0,0,0,0.0813516573054785,0,0,0,2.30772104748742,0,0,0.113462530850897,0,0.0279762322988018,0,0,0,0,0,0,0,3.23601791813763,0,0,0.656722581376586,0,0,0.018708284652282,0,0,0.00664887007716417,0,0,0,0,0.142840481460645,0,0,0,0,0,0,0,0,0,0,0.0234269823295255,0,0,0,0,0,0.1,0.018116327971628,0,0.0534160839712029,0,0,0,0,0,0,0,5.8,0,0,0
1,2,음식,품목대표,닭갈비,구이류,육류구이,400.0,g,-,-,595.61,-,276.4,-,45.9,25.8,

## 2. 타입 변경

In [8]:
food_nutrients_copy = food_nutrients_copy.astype({'1회제공량':'float'})
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].astype(float)

food_nutrients_copy.head()

,NO,DB군,상용제품,식품명,식품대분류,식품상세분류,1회제공량,내용량_단위,총내용량(g),총내용량(mL),에너지(㎉),에너지(kj),수분(g),수분(%),단백질(g),지방(g),지질-가식부 100g당,탄수화물(g),총당류(g),자당(g),포도당(g),과당(g),유당(g),맥아당(g),갈락토오스(g),당알콜(g),에리스리톨(g),총 식이섬유(g),총 식이섬유(mg),총 식이섬유(%),수용성 식이섬유(g),불용성 식이섬유(g),셀룰로오스(%),리그닌(%),칼슘(㎎),철(㎎),철(㎍),마그네슘(㎎),인(㎎),칼륨(g),칼륨(㎎),나트륨(㎎),아연(㎎),구리(㎎),구리(㎍),망간(㎎),망간(㎍),셀레늄(㎍),몰리브덴(㎍),요오드(㎍),염소(㎎),비타민 A(㎍),비타민 A(㎍ RE),레티놀(㎍),베타카로틴(㎍),레티놀A효능,비타민 D(D2+D3)(㎍),비타민 D2(㎍),비타민 D3(㎍),비타민 D1(㎍),비타민 E(㎎),비타민 E(㎎ α-TE),알파 토코페롤(㎎),베타 토코페롤(㎎),감마 토코페롤(㎎),델타 토코페롤(㎎),알파 토코트리에놀(㎎),베타 토코트리에놀(㎎),감마 토코트리에놀(㎎),델타 토코트리에놀(㎎),토코페롤(㎎),토코트리에놀(㎎),비타민 K(㎎),비타민 K(㎍),비타민 K1(㎍),비타민 K2(㎍),비타민 B1(㎎),비타민 B1(㎍),비타민 B2(㎎),비타민 B2(㎍),나이아신(㎎),나이아신(㎎ NE),나이아신(NE)(㎎),나이아신(NE)(㎎ NE),니코틴산 (㎎),니코틴아마이드(㎎),판토텐산(㎎),판토텐산(㎍),비타민 B6(㎎),비타민 B6(㎍),피리독신(㎎),비오틴(㎍),엽산(DFE)(㎍),엽산 - 천연 엽산(㎍),엽산 - 합성 엽산(㎍),비타민 B12(㎎),비타민 B12(㎍),비타민 C(g),비타민 C(㎎),콜린(g),콜린(㎎),총 아미노산(g),총 아미노산(㎎),필수 아미노산(㎎),이소류신(㎎),류신(㎎),라이신(㎎),메티오닌(㎎),페닐알라닌(㎎),트레오닌(㎎),트립토판(㎎),발린(㎎),히스티딘(㎎),아르기닌(㎎),비필수 아미노산(㎎),티로신(㎎),시스테인(㎎),알라닌(㎎),아스파르트산(㎎),글루탐산(㎎),글리신(㎎),프롤린(㎎),세린(㎎),타우린(㎎),글리신 베타인 (㎎),호마린(㎎),트리고넬린(㎎),리보핵산(㎎),데옥시리보핵산(㎎),콜레스테롤(g),콜레스테롤(㎎),총 지방산(g),총 필수 지방산(g),총 포화 지방산(g),총 포화 지방산(%),부티르산(4:0)(g),부티르산(4:0)(㎎),카프로산(6:0)(g),카프로산(6:0)(㎎),카프릴산(8:0)(g),카프릴산(8:0)(㎎),카프르산(10:0)(g),카프르산(10:0)(㎎),라우르산(12:0)(g),라우르산(12:0)(㎎),라우르산(12:0)(%),트라이데칸산(13:0)(㎎),미리스트산(14:0)(g),미리스트산(14:0)(㎎),미리스트산(14:0)(%),펜타데칸산(15:0)(㎎),팔미트산(16:0)(g),팔미트산(16:0)(㎎),팔미트산(16:0)(%),헵타데칸산(17:0)(㎎),스테아르산(18:0)(g),스테아르산(18:0)(㎎),스테아르산(18:0)(%),아라키드산(20:0)(g),아라키드산(20:0)(㎎),아라키드산(20:0)(%),헨에이코산산(21:0)(㎎),베헨산(22:0)(㎎),트리코산산(23:0)(㎎),리그노세르산(24:0)(㎎),총 단일 불포화지방산(g),총 단일 불포화지방산(%),미리스톨레산(14:1)(g),미리스톨레산(14:1)(㎎),미리스톨레산(14:1)(%),팔미톨레산(16:1)(g),팔미톨레산(16:1)(㎎),팔미톨레산(16:1)(%),헵타데센산(17:1)(㎎),올레산(18:1(n-9))(g),올레산(18:1(n-9))(㎎),올레산(18:1(n-9))(%),박센산(18:1(n-7))(g),박센산(18:1(n-7))(㎎),가돌레산(20:1)(g),가돌레산(20:1)(㎎),가돌레산(20:1)(%),에루크산(22:1)(㎎),에루크산(22:1)(%),네르본산(24:1)(㎎),총 다중 불포화지방산(g),총 다중 불포화지방산(%),리놀레산(18:2(n-6)c)(g),리놀레산(18:2(n-6)c)(㎎),리놀레산(18:2(n-6)c)(%),알파 리놀렌산(18:3(n-3))(g),알파 리놀렌산(18:3(n-3))(㎎),리놀렌산(18:3(n-3)(%),감마 리놀렌산(18:3(n-6))(g),감마 리놀렌산(18:3(n-6))(㎎),스테아리돈산(18:4)(%),에이코사디에노산(20:2(n-6))(g),에이코사디에노산(20:2(n-6))(㎎),에이코사트리에노산(20:3(n-3))(㎎),에이코사트리에노산(20:3(n-6))(g),에이코사트리에노산(20:3(n-6))(㎎),아라키돈산(20:4(n-6))(g),아라키돈산(20:4(n-6))(㎎),아라키돈산(20:4(n-6))(%),에이코사테트라에노산(20:4(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(g),에이코사펜타에노산(20:5(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(%),도코사디에노산(22:2)(㎎),도코사펜타에노산(22:5(n-3))(g),도코사펜타에노산(22:5(n-3))(㎎),도코사펜타에노산(22:5(n-3))(%),도코사헥사에노산(22:6(n-3))(g),도코사헥사에노산(22:6(n-3))(㎎),도코사헥사에노산(22:6(n-3))(%),EPA와 DHA의 합(㎎),오메가 3 지방산(g),오메가 6 지방산(g),트랜스 지방산(g),트랜스 올레산(18:1(n-9)t)(g),트랜스 올레산(18:1(n-9)t)(㎎),트랜스 리놀레산 (18:2t)(g),트랜스 리놀레산 (18:2t)(㎎),트랜스 리놀렌산(18:3t)(g),트랜스 리놀렌산(18:3t)(㎎),트랜스 리놀렌산(18:3t)(%),냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),가식부(%),산가용성물질(%),카페인(㎎)
0,1,음식,품목대표,꿩불고기,구이류,육류구이,500.0,g,-,-,368.8,-,412.6,-,33.500000,8.500000,0.0,39.700000,16.900000,7.200000,2.800000,2.800000,0.7,3.500000,0.0,0.0,0.0,9.800000,0.0,0.0,0.0,0.0,0.0,0.0,105.610000,0.0,4.000000,85.39,458.050000,0.0,1243.12000,1264.310000,3.99,0.32,0.0,0.68,0.0,47.55,0.0,0.0,0.0,0.0,0.0,0.000000,1424.580000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.73,0.06,0.0,0.0,0.0,0.0,0.000000,0.0,0.330000,0.0,3.610000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.13,0.0,0.0,0.0,0.00,0.0,2.99,0.0,0.0,0.0,0.0,0.0,1284.375821,2328.598928,2484.750478,645.190070,1274.927303,1410.841278,0.0,1367.688313,848.539312,2205.584777,0.0,919.805669,276.498721,1642.381190,2781.773048,4784.606347,1419.924796,1092.295483,1264.957951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106.180000,0.0,0.0,1.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.007115,0.0,0.0,0.0,0.031784,0.0,0.0,0.0,1.283841,0.0,0.0,0.0,0.527395,0.0,0.0,0.029012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.081352,0.0,0.0,0.0,2.307721,0.0,0.0,0.113463,0.0,0.027976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.236018,0.0,0.0,0.656723,0.0,0.0,0.018708,0.0,0.0,0.006649,0.0,0.0,0.000000,0.0,0.142840,0.0,0.0,0.0,0.000000,0.0,0.0,

## 3. 영양소 별 데이터 단위 통일

In [9]:
gTomg = 1000

food_nutrients_copy.loc[:, [col for col in food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] if col.find('mg')]] / gTomg
food_nutrients_copy.loc[:, [col for col in food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] if col.find('㎍')]] / (gTomg * gTomg)

,단백질(g),지방(g),지질-가식부 100g당,탄수화물(g),총당류(g),자당(g),포도당(g),과당(g),유당(g),맥아당(g),갈락토오스(g),당알콜(g),에리스리톨(g),총 식이섬유(g),총 식이섬유(mg),총 식이섬유(%),수용성 식이섬유(g),불용성 식이섬유(g),셀룰로오스(%),리그닌(%),칼슘(㎎),철(㎎),철(㎍),마그네슘(㎎),인(㎎),칼륨(g),칼륨(㎎),나트륨(㎎),아연(㎎),구리(㎎),구리(㎍),망간(㎎),망간(㎍),셀레늄(㎍),몰리브덴(㎍),요오드(㎍),염소(㎎),비타민 A(㎍),비타민 A(㎍ RE),레티놀(㎍),베타카로틴(㎍),레티놀A효능,비타민 D(D2+D3)(㎍),비타민 D2(㎍),비타민 D3(㎍),비타민 D1(㎍),비타민 E(㎎),비타민 E(㎎ α-TE),알파 토코페롤(㎎),베타 토코페롤(㎎),감마 토코페롤(㎎),델타 토코페롤(㎎),알파 토코트리에놀(㎎),베타 토코트리에놀(㎎),감마 토코트리에놀(㎎),델타 토코트리에놀(㎎),토코페롤(㎎),토코트리에놀(㎎),비타민 K(㎎),비타민 K(㎍),비타민 K1(㎍),비타민 K2(㎍),비타민 B1(㎎),비타민 B1(㎍),비타민 B2(㎎),비타민 B2(㎍),나이아신(㎎),나이아신(㎎ NE),나이아신(NE)(㎎),나이아신(NE)(㎎ NE),니코틴산 (㎎),니코틴아마이드(㎎),판토텐산(㎎),판토텐산(㎍),비타민 B6(㎎),비타민 B6(㎍),피리독신(㎎),비오틴(㎍),엽산(DFE)(㎍),엽산 - 천연 엽산(㎍),엽산 - 합성 엽산(㎍),비타민 B12(㎎),비타민 B12(㎍),비타민 C(g),비타민 C(㎎),콜린(g),콜린(㎎),총 아미노산(g),총 아미노산(㎎),필수 아미노산(㎎),이소류신(㎎),류신(㎎),라이신(㎎),메티오닌(㎎),페닐알라닌(㎎),트레오닌(㎎),트립토판(㎎),발린(㎎),히스티딘(㎎),아르기닌(㎎),비필수 아미노산(㎎),티로신(㎎),시스테인(㎎),알라닌(㎎),아스파르트산(㎎),글루탐산(㎎),글리신(㎎),프롤린(㎎),세린(㎎),타우린(㎎),글리신 베타인 (㎎),호마린(㎎),트리고넬린(㎎),리보핵산(㎎),데옥시리보핵산(㎎),콜레스테롤(g),콜레스테롤(㎎),총 지방산(g),총 필수 지방산(g),총 포화 지방산(g),총 포화 지방산(%),부티르산(4:0)(g),부티르산(4:0)(㎎),카프로산(6:0)(g),카프로산(6:0)(㎎),카프릴산(8:0)(g),카프릴산(8:0)(㎎),카프르산(10:0)(g),카프르산(10:0)(㎎),라우르산(12:0)(g),라우르산(12:0)(㎎),라우르산(12:0)(%),트라이데칸산(13:0)(㎎),미리스트산(14:0)(g),미리스트산(14:0)(㎎),미리스트산(14:0)(%),펜타데칸산(15:0)(㎎),팔미트산(16:0)(g),팔미트산(16:0)(㎎),팔미트산(16:0)(%),헵타데칸산(17:0)(㎎),스테아르산(18:0)(g),스테아르산(18:0)(㎎),스테아르산(18:0)(%),아라키드산(20:0)(g),아라키드산(20:0)(㎎),아라키드산(20:0)(%),헨에이코산산(21:0)(㎎),베헨산(22:0)(㎎),트리코산산(23:0)(㎎),리그노세르산(24:0)(㎎),총 단일 불포화지방산(g),총 단일 불포화지방산(%),미리스톨레산(14:1)(g),미리스톨레산(14:1)(㎎),미리스톨레산(14:1)(%),팔미톨레산(16:1)(g),팔미톨레산(16:1)(㎎),팔미톨레산(16:1)(%),헵타데센산(17:1)(㎎),올레산(18:1(n-9))(g),올레산(18:1(n-9))(㎎),올레산(18:1(n-9))(%),박센산(18:1(n-7))(g),박센산(18:1(n-7))(㎎),가돌레산(20:1)(g),가돌레산(20:1)(㎎),가돌레산(20:1)(%),에루크산(22:1)(㎎),에루크산(22:1)(%),네르본산(24:1)(㎎),총 다중 불포화지방산(g),총 다중 불포화지방산(%),리놀레산(18:2(n-6)c)(g),리놀레산(18:2(n-6)c)(㎎),리놀레산(18:2(n-6)c)(%),알파 리놀렌산(18:3(n-3))(g),알파 리놀렌산(18:3(n-3))(㎎),리놀렌산(18:3(n-3)(%),감마 리놀렌산(18:3(n-6))(g),감마 리놀렌산(18:3(n-6))(㎎),스테아리돈산(18:4)(%),에이코사디에노산(20:2(n-6))(g),에이코사디에노산(20:2(n-6))(㎎),에이코사트리에노산(20:3(n-3))(㎎),에이코사트리에노산(20:3(n-6))(g),에이코사트리에노산(20:3(n-6))(㎎),아라키돈산(20:4(n-6))(g),아라키돈산(20:4(n-6))(㎎),아라키돈산(20:4(n-6))(%),에이코사테트라에노산(20:4(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(g),에이코사펜타에노산(20:5(n-3))(㎎),에이코사펜타에노산(20:5(n-3))(%),도코사디에노산(22:2)(㎎),도코사펜타에노산(22:5(n-3))(g),도코사펜타에노산(22:5(n-3))(㎎),도코사펜타에노산(22:5(n-3))(%),도코사헥사에노산(22:6(n-3))(g),도코사헥사에노산(22:6(n-3))(㎎),도코사헥사에노산(22:6(n-3))(%),EPA와 DHA의 합(㎎),오메가 3 지방산(g),오메가 6 지방산(g),트랜스 지방산(g),트랜스 올레산(18:1(n-9)t)(g),트랜스 올레산(18:1(n-9)t)(㎎),트랜스 리놀레산 (18:2t)(g),트랜스 리놀레산 (18:2t)(㎎),트랜스 리놀렌산(18:3t)(g),트랜스 리놀렌산(18:3t)(㎎),트랜스 리놀렌산(18:3t)(%),냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),가식부(%),산가용성물질(%),카페인(㎎)
0,0.000034,0.000008,0.0,0.000040,0.000017,7.200000e-06,2.800000e-06,0.000003,7.000000e-07,3.500000e-06,0.0,0.0,0.0,0.000010,0.0,0.0,0.0,0.0,0.0,0.0,0.000106,0.0,0.000004,0.000085,0.000458,0.0,0.001243,0.001264,3.990000e-06,3.200000e-07,0.0,6.800000e-07,0.0,0.000048,0.0,0.0,0.0,0.0,0.0,0.000000,0.001425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000009,6.000000e-08,0.0,0.0,0.0,0.0,0.000000e+00,0.0,3.300000e-07,0.0,3.610000e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000063,0.0,0.0,0.0,0.000000,0.0,0.000003,0.0,0.0,0.0,0.0,0.0,0.001284,0.002329,0.002485,0.000645,0.001275,0.001411,0.0,0.001368,0.000849,0.002206,0.0,0.000920,0.000276,0.001642,0.002782,0.004785,0.001420,0.001092,0.001265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.061800e-04,0.0,0.0,1.900000e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,7.114835e-09,0.0,0.0,0.0,3.178355e-08,0.0,0.0,0.0,1.283841e-06,0.0,0.0,0.0,5.273953e-07,0.0,0.0,2.901235e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,8.135166e-08,0.0,0.0,0.0,0.000002,0.0,0.0,1.134625e-07,0.0,2.797623e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000003,0.0,0.0,6.567226e-07,0.0,0.0,1.870828e-08,0.0,0.0,6.648870e-09,0.0,0.0,0.000000e+00,0.0,1.428405e-07,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,2.342698e-08,0.0,0.

##### 영향소별 비율을 구하기 위해 데이터를 살펴본 결과 데이터가 복잡.?
ex)아라키돈산, 에이코사펜타에노산 같은 영양소 데이터의 경우 g, mg, %로 나뉘어져 있으며 식품에 따라 표기된 위치가 다르다 다행히 g에도 적혀있고 mg에도 적히는 등 중복으로 된 것은 없어서 통합하면 될 것 같다. % 에 표기된 경우 연산 없이 바로 %값을 사용 -> %값이 있는 데이터의 경우 연산을 진행할 때 제외하고 한다....
